# Parte 3: Estudio de diferentes algoritmos de machine learning

En este notebook vamos a aplicar diferentes algoritmos para estudiar su efectividad para este problema e intentaremos predicciones para los Juegos Olímpicos de 2021.

Los algoritmos que se aplican son: 
* Matriz de confusión
* Datos de entrenamiento y datos de test
* Knn - Vecinos cercanos
* Regresión Lineal
* Regresión Logística
* Naive Bayes
* Gaussian Naive Bayes
* K-means
* Perceptron
* MLP - Multi layer perceptron
* Decision Tree
* SVM - Supervised Vector Machine
* Descenso del gradiente
* Random Forest
* ¿¿RED NEURONAL??

In [1]:
import pandas as pd

import numpy as np
from numpy import * 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lee el excel generado en el otro documento .ipynb
ddbb=pd.read_excel("/Github/DecatlonEstadistics/resources/data.xlsx")
del ddbb['1500m NF']

ddbb['Country']=ddbb['Country'].str.lower()

ddbb.columns = ['Position', 'Athlete', 'Age', 'Country', 'Total Points', 'Year', 'Competition', '100m', 
                 '100m Points', 'Lj', 'Lj Points', 'Sp', 'Sp Points', 'Hj', 'Hj Points', '400m', '400m Points', 
                 '110m H', '110m H Points', 'Dt', 'Dt Points', 'Pv', 'Pv Points', 'Jt', 'Jt Points', 
                 '1500m Points', '1500m']

## 0. Métricas Algoritmos

In [3]:
def metricasAlgorithms(y_test, y_pred):
    metricas = []
    
    metricas.append(accuracy_score(y_test, y_pred))
    
    metricas.append(precision_score(y_test, y_pred, average='micro'))
    metricas.append(recall_score(y_test, y_pred, average='micro'))
    metricas.append(f1_score(y_test, y_pred, average='micro'))

    metricas.append(precision_score(y_test, y_pred, average='macro'))
    metricas.append(recall_score(y_test, y_pred, average='macro'))
    metricas.append(f1_score(y_test, y_pred, average='macro'))

    metricas.append(precision_score(y_test, y_pred, average='weighted'))
    metricas.append(recall_score(y_test, y_pred, average='weighted'))
    metricas.append(f1_score(y_test, y_pred, average='weighted'))
    
    return metricas

def metricasAlgorithmsRegl(X_test, y_test, regL):
    metricas = []
    
    metricas.append(regL.score(X_test,y_test))
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    
    return metricas

In [4]:
pKmeans = []

for i in range(len(ddbb)):
    if(ddbb.iloc[i]['Position'] <= 10):
        pKmeans.append(10)
    elif(ddbb.iloc[i]['Position'] > 10 and ddbb.iloc[i]['Position'] <= 25):
        pKmeans.append(25)
    elif(ddbb.iloc[i]['Position'] > 25):
        pKmeans.append(30)

## 2. Vecinos Cercanos

In [5]:
def knnFunction(X_train, X_test, y_train, y_test):
    
    knn = KNeighborsClassifier(n_neighbors=5)

    #Entrena el modelo
    knn.fit(X_train, y_train)

    #Predice para los datos de test
    y_pred = knn.predict(X_test)
    
    vector = metricasAlgorithms(y_test, y_pred)
    
    return(vector)

## 3. Regresión Lineal

In [6]:
def RegLinealFunction(X_train, X_test, y_train, y_test):
    
    regL = linear_model.LinearRegression()
    
    regL.fit(X_train, y_train)
    
    y_pred = regL.predict(X_test)
    
    return metricasAlgorithmsRegl(X_test, y_test, regL)

## 4. Regresión Logística

In [7]:
def RegLogisFunction(X_train, X_test, y_train, y_test):
    
    logreg = linear_model.LogisticRegression(random_state=0)

    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    
    return metricasAlgorithms(y_test, y_pred)

## 5. Naive Bayes

In [8]:
def NaiveBayesFunction(X_train, X_test, y_train, y_test):
    
    gnb = GaussianNB()

    gnb.fit(X_train, y_train)

    y_pred = gnb.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

##  6. K-Means

In [9]:
def KmeansFunction(pKmeans, ddbbData):

    X = scale(ddbbData)
    y = pd.DataFrame(pKmeans)

    kmeans1 = KMeans(n_clusters=1 , random_state= 5)

    kmeans1.fit(X)

    y_pred = np.choose(kmeans1.labels_,[10,25,30]).astype(np.int64)

    return metricasAlgorithms(y, y_pred)

## 7. Perceptron

In [10]:
def PerceptronFunction(X_train, X_test, y_train, y_test):
    
    percep = Perceptron(tol=1e-5, random_state=1)

    percep.fit(X_train, y_train)

    y_pred = percep.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 8. MLP- Perceptron Multi Capa

In [11]:
def MLPFunction(X_train, X_test, y_train, y_test):

    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 9. SVM - Supervised Vector Machine

In [12]:
def SVMFunction(X_train, X_test, y_train, y_test):

    clf = svm.SVC(kernel='linear')

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

## 10. Descenso del gradiente

In [13]:
def DescensoGradFunction(X_train, X_test, y_train, y_test):

    dGrad = SGDClassifier(loss="hinge", penalty="l2")

    dGrad.fit(X_train, y_train)

    y_pred = dGrad.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

## 11. Arbol de decision

In [14]:
def ArbolDecFunction(X_train, X_test, y_train, y_test):

    clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

    clf = clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 12. Random Forest

In [15]:
def BosqueAlFunction(X_train, X_test, y_train, y_test):

    rForest=RandomForestClassifier(n_estimators=100)

    rForest.fit(X_train,y_train)

    y_pred=rForest.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

In [16]:
def ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test):
    knn = knnFunction(X_train, X_test, y_train, y_test)
    #kmeans = KmeansFunction(pKmeans, ddbbData)
    RegLin = RegLinealFunction(X_train, X_test, y_train, y_test)
    RegLog = RegLogisFunction(X_train, X_test, y_train, y_test)
    NBayes = NaiveBayesFunction(X_train, X_test, y_train, y_test)
    Percep = PerceptronFunction(X_train, X_test, y_train, y_test)
    DGrad = DescensoGradFunction(X_train, X_test, y_train, y_test)
    mlp = MLPFunction(X_train, X_test, y_train, y_test)
    ADecis = ArbolDecFunction(X_train, X_test, y_train, y_test)
    BAleatorio = BosqueAlFunction(X_train, X_test, y_train, y_test)
    dataFrameDatos = pd.DataFrame({
                                    'Metricas': ['Accuracy','Precision Micro','Recall Micro','F1 Micro',
                                                 'Precision Macro','Recall Macro','F1 Macro','Precision Weigthed',
                                                 'Recall Weigthed','F1 Weigthed'],
                                    'knn': knn,
                                    'Regresion lineal': RegLin,
                                    'Regresion logistica': RegLog,
                                    #'Kmeans' : kmeans,
                                    'Naive bayes': NBayes,
                                    'Perceptron': Percep,
                                    'MLP': mlp,
                                    #'SVM': SVMFunction(X_train, X_test, y_train, y_test),
                                    'Descenso gradiente': DGrad,
                                    'Arbol de decision': ADecis,
                                    'Bosque aleatorios': BAleatorio 
                                })
    return (dataFrameDatos.loc[[0]])

## 1. Datos de entrenamiento y datos de test.

Los datos se entrenan en función a todas las pruebas resultantes y se clasifican en función a la posición obtenida.
El 70% de los datos se utilizan para entrenamiento.
El 30% restante se utiliza para tests.

In [17]:
listaResults = []

In [18]:
# Ejecucion para 100m
ddbbData = ddbb[['100m','100m Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.062745,0.214722,0.054902,0.082353,0.047059,0.035294,0.003922,0.062745,0.066667


In [19]:
# Ejecucion para Lj
ddbbData = ddbb[['Lj','Lj Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.043137,0.067461,0.035294,0.043137,0.043137,0.047059,0.043137,0.047059,0.039216


In [20]:
# Ejecucion para Sp
ddbbData = ddbb[['Sp','Sp Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.047059,0.076245,0.039216,0.070588,0.027451,0.035294,0.027451,0.035294,0.019608


In [21]:
# Ejecucion para Hj
ddbbData = ddbb[['Hj','Hj Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.019608,0.092699,0.027451,0.05098,0.003922,0.027451,0.023529,0.05098,0.031373


In [22]:
# Ejecucion para 400m
ddbbData = ddbb[['400m Points','400m']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.035294,0.064629,0.047059,0.047059,0.011765,0.035294,0.035294,0.035294,0.05098


In [23]:
# Ejecucion para 110m H
ddbbData = ddbb[['110m H','110m H Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.054902,0.021908,0.043137,0.031373,0.043137,0.031373,0.043137,0.043137,0.047059


In [24]:
# Ejecucion para Dt
ddbbData = ddbb[['Dt','Dt Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.039216,0.118879,0.043137,0.058824,0.054902,0.027451,0.054902,0.062745,0.031373


In [25]:
# Ejecucion para Pv
ddbbData = ddbb[['Pv','Pv Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.062745,-0.009883,0.035294,0.035294,0.039216,0.039216,0.047059,0.047059,0.047059


In [26]:
# Ejecucion para Jt
ddbbData = ddbb[['Jt','Jt Points']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.047059,0.11022,0.039216,0.047059,0.031373,0.027451,0.043137,0.070588,0.062745


In [27]:
# Ejecucion para 1500m
ddbbData = ddbb[['1500m Points','1500m']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

result = ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)
listaResults.append(result)
result

,Metricas,knn,Regresion lineal,Regresion logistica,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy,0.058824,0.026264,0.039216,0.027451,0.039216,0.035294,0.043137,0.039216,0.043137


In [28]:
mediaPrueba = []

for i in range(len(listaResults)):
    for j in range(len(listaResults[0].iloc[0])):
        if(j != 0):
            mediaPrueba[i] += listaResults[i].iloc[0][j]
        else:
            mediaPrueba.append(0)

print(mediaPrueba)

[0.6304079438157099, 0.40863742867930397, 0.3782058684668354, 0.32799359864543115, 0.3626678515135463, 0.3591624600029465, 0.49142787073196204, 0.3430583839204366, 0.4788473024969301, 0.35175374814952837]


### Las pruebas más importantes, según las medias son:
- 100m: 0.51
- 400m: 0.48
- Javelin Throw: 0.56

# 13.Predicciones

In [45]:
ddbbData = ddbb[['100m', '100m Points', '400m', '400m Points', 'Jt', 'Jt Points']]

#Creamos el dataset para entrenar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

#Entrenamos el modelo
ejecutaAlgoritmosMachineLearning(X_train, X_test, y_train, y_test)

#Crea el dataset con los datos para las predicciones
predictionData = pd.read_csv("/Github/DecatlonEstadistics/resources/testResults.csv")
del predictionData['Unnamed: 7']

predictionData.columns = ['Athlete','100m','100m Points','400m','400m Points','Jt','Jt Points']

predDataset = predictionData[['100m','100m Points','400m','400m Points','Jt','Jt Points']]

X_test = predDataset

#Predicciones Regresion Lineal
regL = linear_model.LinearRegression()
    
regL.fit(X_train, y_train)
    
y_pred = regL.predict(X_test)

 = [int(x) for x in a]

predictionData.insert(7, "Postion Predicted", y_pred, True)
predictionData

TypeError: only integer scalar arrays can be converted to a scalar index